In [15]:
import os
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import gaussian_filter
import pickle
import numpy as np

In [16]:
def load_objects_from_file(filename, subfolder=""):
    # Adjust the filename to include the subfolder
    if subfolder:
        filename = os.path.join(subfolder, filename)
        
    # Load the objects from file
    with open(filename, 'rb') as f:
        objects_list = pickle.load(f)
    
    return objects_list

In [17]:
redwood_circle_objects_branch_orig = load_objects_from_file("redwood_circle_objects_orig.pkl", subfolder="object_data")
redwood_circle_objects_branch_origB10th = load_objects_from_file("redwood_circle_objects_origB10th.pkl", subfolder="object_data")
guiana_circle_objects_leaf_orig = load_objects_from_file("guiana_circle_objects_leaf_orig.pkl", subfolder="object_data")
guiana_circle_objects_leaf_div10 = load_objects_from_file("guiana_circle_objects_leaf_div10.pkl", subfolder="object_data")
guiana_circle_objects_leaf_div30 = load_objects_from_file("guiana_circle_objects_leaf_div30.pkl", subfolder="object_data")

In [19]:
def return_waveform_array(circle_name, pdf_path):

    subfolder = pdf_path.split('/')[-2]
    leaf_branch_ident = pdf_path.split('/')[-1]

    if subfolder == 'pro_fg_orig':
        circle_objects = guiana_circle_objects_leaf_orig
    elif subfolder == 'pro_fg_origLD10th':
        circle_objects = guiana_circle_objects_leaf_div10
    elif subfolder == 'pro_fg_origLD30th':
        circle_objects = guiana_circle_objects_leaf_div30
    elif subfolder == 'pro_rw_orig':
        circle_objects = redwood_circle_objects_branch_orig
    elif subfolder == 'pro_rw_B10th':
        circle_objects = redwood_circle_objects_branch_origB10th
    else:
        raise ValueError(f"Unsupported site_version: {subfolder}")

    # Determine use_branch based on leaf_branch_ident
    if '_fb_' in leaf_branch_ident:
        use_branch = True
    elif '_f_' in leaf_branch_ident:
        use_branch = False
    else:
        raise ValueError(f"Unsupported leaf_branch_ident: {leaf_branch_ident}")
    
    # Find the correct circle object
    for circle in circle_objects:
        if circle_name == circle.circle_name:
            height_array = circle.tree_height_array.tolist()
            if use_branch:
                return height_array, circle.waveform_b.tolist()
            else:
                return height_array, circle.waveform.tolist()
            
    raise ValueError(f"{circle_name} not found in {subfolder} circle objects.")

def overlay_plot_on_image(image, height_array, waveform, circle_name, position, fig_size=(3, 3), alpha=0.5):
    fig, ax = plt.subplots(figsize=fig_size)

    dz = 0.1
    n_ext = int(10.0 / dz)

    new_gort_ght = np.zeros(len(waveform) + n_ext, dtype=np.float32)
    new_gort_waveform_ext = np.zeros(len(waveform) + n_ext, dtype=np.float32)
    new_gort_waveform_ext[:n_ext] = 0  # extend wvfm to below ground 10m deep

    for new_gort_ilevel in range(len(waveform)):
        new_gort_ght[new_gort_ilevel + n_ext] = height_array[new_gort_ilevel]
        new_gort_waveform_ext[new_gort_ilevel + n_ext] = waveform[new_gort_ilevel]

    width = int(0.6 / dz) + 1
    new_gort_normalized_waveform = gaussian_filter(new_gort_waveform_ext, sigma = width, mode = 'nearest')

    ax.plot(new_gort_normalized_waveform, new_gort_ght, label=circle_name, color='red')
    ax.set_title(circle_name)
    ax.set_xlabel('Waveform')
    ax.set_ylabel('Height (m)')
    ax.legend()

    # Save the plot as an image
    plot_path = "temp_plot.png"
    fig.savefig(plot_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    # Open the plot image
    plot_image = Image.open(plot_path).convert("RGBA")
    
    # Resize the plot image
    plot_image = plot_image.resize((fig_size[0] * 100, fig_size[1] * 100), Image.LANCZOS)

    # Create an alpha mask
    alpha_mask = Image.new("L", plot_image.size, int(255 * alpha))
    plot_image.putalpha(alpha_mask)
    
    # Get the original image size
    width, height = image.size
    
    # Calculate the position to place the plot
    x_offset = position[1] * (width // 3)
    y_offset = position[0] * (height // 3)
    
    # Composite the images
    image.paste(plot_image, (x_offset, y_offset), plot_image)

def generate_plots_with_overlay(pdf_path, output_image_path):
    # Ensure the output path exists
    os.makedirs(os.path.dirname(output_image_path), exist_ok=True)

    circle_order = [
        ['Circle(1, 1)', 'Circle(1, 2)', 'Circle(1, 3)'],
        ['Circle(2, 1)', 'Circle(2, 2)', 'Circle(2, 3)'],
        ['Circle(3, 1)', 'Circle(3, 2)', 'Circle(3, 3)']
    ]

    # Convert the single-page PDF to an image
    image = convert_from_path(pdf_path)[0]
    image = image.convert("RGBA")

    # Overlay plots on the image
    for row in range(3):
        for col in range(3):
            circle_name = circle_order[row][col]
            height_array, waveform = return_waveform_array(circle_name, pdf_path)
            overlay_plot_on_image(image, height_array, waveform, circle_name, (row, col))

    # Save the final image
    image.save(output_image_path, "PNG")

    print(f"Overlay plots saved to {output_image_path}")

# Example usage:
pdf_path = "C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/gort_model/rw/pro_rw_orig/rw_f_wvfm.pdf"
pdf_b_path = "C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/gort_model/rw/pro_rw_orig/rw_fb_wvfm.pdf"
output_image_path = "C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/overlay_GORT/combined_plot.png"
overlay_data = ([0.2, 0.4, 0.6, 0.8], [0.2, 0.5, 0.7, 0.9])

generate_plots_with_overlay(pdf_path, output_image_path)

Overlay plots saved to C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/overlay_GORT/combined_plot.png
